# Location

The `location` directive, used within the `server` context, specifies how NGINX should process requests based on the URI.

More in [correspoding section](http://nginx.org/en/docs/http/ngx_http_core_module.html#location) of the official documentation.

## Setup

It turns out that it's quite a complex task to build examples that show how everything works, so this section describes what we need to show everything. In summary, we need

- Proxied server - server where we'll redirect requests to nginx.
- And nginx, which can be configured differently for different examples.
- A network that connects containers.

The next cell will create all the necessary ones.

In [1]:
docker network create test_network

docker run --rm -d -p 81:80 \
    --name client_container \
    --network test_network \
    kennethreitz/httpbin

docker run -p 80:80 -d --rm \
    --network test_network \
    --name experiment_nginx nginx

14760b5ff2a9b2a0dfd6e7effc9a3db700d94ab71faa1c2e331dec51c379db22
fba0bc36ed733193988814c0bc51f08a95ccf5140446cf66fe52302c6f0265fc
e5117e20eb07f9dae9b25ad798114cfd0ae70eaceaa5f89c0c257d9779547f61


**Note** Don't forget to clear the environment.

In [9]:
docker stop client_container experiment_nginx
docker network rm test_network

client_container
experiment_nginx
test_network


## URI matching

The `location` directive has an argument that specifies rules for which URI to match in which context. This section considers several options.

---

In the following example, nginx config defines two locations:  `/pattern' and `/pattern/` URIs. By looking at the output, you can see exactly which location was triggered. They're almost the same, but you'll see that they match the different cases.

In [71]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' << EOF
events {}
http {
    server {
        listen 80;
        
        location /pattern {
            return 200 "location 1 - I use /pattern";
        }

        location /pattern/ {
            return 200 "location 2 - I use /pattern/";
        }
    }
}
EOF

docker exec -it experiment_nginx nginx -s reload

2024/07/23 14:04:27 [notice] 434#434: signal process started


Now we can request different URIs from nginx and see what exactly has been triggered.

The following example shows what happens when you access `/pattern` (no `/` at the end of the command).

In [72]:
curl -L http://localhost:80/pattern

location 1 - I use /pattern


The next cell shows what happens when you access `/pattern/wow` (we use an additional path segment).

In [ ]:
curl -L http://localhost:80/pattern/wow

you are on the /pattern/


## Headers to server (`proxy_set_header`)

Allows redefining or appending fields to the request header passed to the proxied server. You can add aditional headers to the http request that will be sent to the destination server. So by using syntax `proxy_set_header <header field> <value>;`.

The following example changes the nginx config to add two new fields to the http header `Name` and `SecondName` and reloads nginx.

In [2]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' << EOF
events {}
http {
    server {
        listen 80;
        location / {
            proxy_pass "http://client_container/headers";
            proxy_set_header Name Fedor;
            proxy_set_header SecondName Kobak;
        }
    }
}
EOF

docker exec -it experiment_nginx nginx -s reload

2024/07/23 11:24:34 [notice] 68#68: signal process started


First, consider what happens if we just request httpbin directly.

In [3]:
curl http://localhost:81/headers

{
  "headers": {
    "Accept": "*/*", 
    "Host": "localhost:81", 
    "User-Agent": "curl/7.81.0"
  }
}


There are no additional headers - just the very basic ones generated by `curl`.

In [4]:
curl http://localhost:80

{
  "headers": {
    "Accept": "*/*", 
    "Connection": "close", 
    "Host": "client_container", 
    "Name": "Fedor", 
    "Secondname": "Kobak", 
    "User-Agent": "curl/7.81.0"
  }
}


If you compare the output of the original and proxied requests, you can see that the proxied requests have additional headers - just as we specified in the nginx configuration.